# 第五课：带记忆的多轮对话智能体

## 学习目标
1. 理解 Checkpointer（检查点）机制
2. 实现多轮对话记忆
3. 构建一个完整的、可以记住上下文的智能体

## 为什么需要记忆？

之前的智能体每次调用都是**独立的**，不记得之前的对话。

有了记忆，智能体可以：
- 📝 记住用户说过的话
- 🧠 理解上下文，给出更好的回答
- 🔄 跨多轮对话完成复杂任务

## 准备工作

In [ ]:
import os
from datetime import datetime
from typing import TypedDict, Annotated, Sequence
import operator

from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage, SystemMessage

# LangGraph 组件
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode, create_react_agent
from langgraph.checkpoint.memory import MemorySaver

# 设置 API Key
os.environ["DASHSCOPE_API_KEY"] = "your_api_key_here"

def create_qwen_chat():
    api_key = os.getenv("DASHSCOPE_API_KEY")
    if not api_key or api_key == "your_api_key_here":
        raise ValueError("⚠️ 请设置你的 DASHSCOPE_API_KEY")
    
    return ChatOpenAI(
        model="qwen-plus",
        openai_api_key=api_key,
        openai_api_base="https://dashscope.aliyuncs.com/compatible-mode/v1",
        temperature=0.7,
    )

print("✅ 准备完成")

---
## 1. 理解 Checkpointer（检查点）

**Checkpointer** 是 LangGraph 的核心概念：
- 保存图执行过程中的**状态快照**
- 每个状态快照用一个唯一的 `thread_id` 标识
- 下次调用时，传入相同的 `thread_id` 就能恢复之前的状态

In [ ]:
print("""
📚 Checkpointer 的工作原理：

    对话 1 (thread_id="user_123")        对话 2 (thread_id="user_456")
    ┌─────────────────────┐              ┌─────────────────────┐
    │ 用户: 我叫小明       │              │ 用户: 你好           │
    │ AI: 你好小明！       │              │ AI: 你好！           │
    │ ─────────────────   │              │ ─────────────────   │
    │ 用户: 我叫什么？     │              │ 用户: 我叫什么？     │
    │ AI: 你叫小明。  ✓    │              │ AI: 你还没告诉我  ✗  │
    └─────────────────────┘              └─────────────────────┘

    每个 thread_id 有独立的对话历史！
    
常用的 Checkpointer 类型：
1. MemorySaver - 内存存储，简单快速，重启后丢失
2. SqliteSaver - SQLite 存储，持久化到本地文件
3. PostgresSaver - PostgreSQL 存储，适合生产环境
""")

---
## 2. 定义工具

In [ ]:
@tool
def get_current_time() -> str:
    """获取当前的日期和时间。"""
    return datetime.now().strftime("%Y年%m月%d日 %H:%M:%S")


@tool
def calculate(expression: str) -> str:
    """计算数学表达式。"""
    try:
        allowed_chars = set("0123456789+-*/.() ")
        if not all(c in allowed_chars for c in expression):
            return "错误：表达式包含不允许的字符"
        result = eval(expression)
        return f"{expression} = {result}"
    except Exception as e:
        return f"计算错误：{str(e)}"


@tool
def get_weather(city: str) -> str:
    """获取指定城市的天气信息。"""
    weather_data = {
        "北京": {"temp": 15, "condition": "晴"},
        "上海": {"temp": 18, "condition": "多云"},
        "广州": {"temp": 25, "condition": "小雨"},
        "深圳": {"temp": 26, "condition": "阴"},
    }
    
    if city in weather_data:
        data = weather_data[city]
        return f"{city}：{data['condition']}，{data['temp']}°C"
    return f"暂无 {city} 的天气数据"


tools = [get_current_time, calculate, get_weather]
print(f"✅ 已定义 {len(tools)} 个工具")

---
## 3. 构建带记忆的智能体

关键步骤：
1. 创建 `MemorySaver` 实例
2. 在编译图时传入 `checkpointer=memory`
3. 调用时传入 `config` 指定 `thread_id`

In [ ]:
# 创建模型
model = create_qwen_chat()

# 🔑 关键：创建 MemorySaver
memory = MemorySaver()

# 使用预构建的 ReAct Agent，并添加记忆
agent = create_react_agent(
    model=model,
    tools=tools,
    checkpointer=memory,  # 🔑 添加记忆
    state_modifier="""你是一个友好的智能助手，具有以下特点：
1. 你会记住用户在对话中提到的信息
2. 你可以使用工具来帮助用户
3. 你的回答简洁友好

可用工具：
- get_current_time: 获取当前时间
- calculate: 数学计算
- get_weather: 查询天气"""
)

print("✅ 带记忆的智能体创建完成！")

---
## 4. 多轮对话演示

关键：使用相同的 `thread_id` 保持对话连续性。

In [ ]:
def chat(message: str, thread_id: str = "default"):
    """发送消息并获取回复"""
    config = {"configurable": {"thread_id": thread_id}}
    
    result = agent.invoke(
        {"messages": [HumanMessage(content=message)]},
        config=config  # 🔑 传入配置
    )
    
    return result["messages"][-1].content

print("✅ chat 函数定义完成")

In [ ]:
# 开始对话（使用同一个 thread_id）
thread_id = "conversation_001"

print("🗣️ 多轮对话演示")
print("=" * 50)
print(f"Thread ID: {thread_id}")
print("-" * 50)

In [ ]:
# 第一轮：自我介绍
msg = "你好！我叫张三，今年 25 岁。"
print(f"\n👤 用户: {msg}")
response = chat(msg, thread_id)
print(f"🤖 AI: {response}")

In [ ]:
# 第二轮：分享更多信息
msg = "我喜欢编程和阅读。"
print(f"\n👤 用户: {msg}")
response = chat(msg, thread_id)
print(f"🤖 AI: {response}")

In [ ]:
# 第三轮：测试记忆 - 名字
msg = "你还记得我叫什么名字吗？"
print(f"\n👤 用户: {msg}")
response = chat(msg, thread_id)
print(f"🤖 AI: {response}")

In [ ]:
# 第四轮：测试记忆 - 年龄和爱好
msg = "我的年龄和爱好呢？"
print(f"\n👤 用户: {msg}")
response = chat(msg, thread_id)
print(f"🤖 AI: {response}")

In [ ]:
# 第五轮：使用工具
msg = "帮我查一下北京的天气。"
print(f"\n👤 用户: {msg}")
response = chat(msg, thread_id)
print(f"🤖 AI: {response}")

In [ ]:
# 第六轮：结合上下文
msg = "如果我去北京旅行，根据天气你有什么建议？"
print(f"\n👤 用户: {msg}")
response = chat(msg, thread_id)
print(f"🤖 AI: {response}")

---
## 5. 不同 thread_id 是独立的

使用不同的 `thread_id`，对话历史是完全独立的。

In [ ]:
# 新的对话（不同的 thread_id）
new_thread = "conversation_002"

print("\n" + "=" * 50)
print(f"新对话 - Thread ID: {new_thread}")
print("-" * 50)

msg = "你知道我叫什么名字吗？"
print(f"\n👤 用户: {msg}")
response = chat(msg, new_thread)
print(f"🤖 AI: {response}")

---
## 6. 查看对话历史

可以使用 `get_state` 方法查看保存的对话状态。

In [ ]:
# 查看对话历史
config = {"configurable": {"thread_id": thread_id}}
state = agent.get_state(config)

print(f"📋 Thread '{thread_id}' 的对话历史:")
print(f"消息数量: {len(state.values['messages'])}")
print("-" * 50)

for i, msg in enumerate(state.values["messages"]):
    if isinstance(msg, HumanMessage):
        print(f"[{i}] 👤 用户: {msg.content}")
    elif isinstance(msg, AIMessage):
        if msg.content:
            content = msg.content[:60] + "..." if len(msg.content) > 60 else msg.content
            print(f"[{i}] 🤖 AI: {content}")
        if msg.tool_calls:
            print(f"[{i}] 🔧 调用: {[tc['name'] for tc in msg.tool_calls]}")
    elif isinstance(msg, ToolMessage):
        print(f"[{i}] 📋 工具: {msg.content}")

---
## 7. 封装成一个智能助手类

将智能体封装成一个易用的类。

In [ ]:
class SmartAssistant:
    """
    智能助手类
    
    特点：
    - 多轮对话记忆
    - 工具调用
    - 简洁的 API
    """
    
    def __init__(self, name: str = "小智"):
        self.name = name
        self.model = create_qwen_chat()
        self.tools = [get_current_time, calculate, get_weather]
        self.memory = MemorySaver()
        
        self.agent = create_react_agent(
            model=self.model,
            tools=self.tools,
            checkpointer=self.memory,
            state_modifier=f"""你是一个智能助手，名叫"{name}"。
友好、热情、简洁地回答用户问题。
会记住用户分享的信息，可以使用工具帮助用户。"""
        )
    
    def chat(self, message: str, session_id: str = "default") -> str:
        """
        发送消息并获取回复
        
        Args:
            message: 用户消息
            session_id: 会话 ID，相同 ID 共享对话历史
        
        Returns:
            AI 的回复
        """
        config = {"configurable": {"thread_id": session_id}}
        result = self.agent.invoke(
            {"messages": [HumanMessage(content=message)]},
            config=config
        )
        return result["messages"][-1].content
    
    def get_history(self, session_id: str = "default") -> list:
        """获取对话历史"""
        config = {"configurable": {"thread_id": session_id}}
        state = self.agent.get_state(config)
        return state.values.get("messages", [])
    
    def get_history_summary(self, session_id: str = "default") -> str:
        """获取对话历史摘要"""
        messages = self.get_history(session_id)
        summary = []
        for msg in messages:
            if isinstance(msg, HumanMessage):
                summary.append(f"用户: {msg.content}")
            elif isinstance(msg, AIMessage) and msg.content:
                summary.append(f"AI: {msg.content[:50]}...")
        return "\n".join(summary)

print("✅ SmartAssistant 类定义完成")

In [ ]:
# 创建助手实例
assistant = SmartAssistant(name="小助")

print(f"🤖 {assistant.name} 已准备就绪！")

In [ ]:
# 使用助手进行对话
session = "user_lihua"

messages = [
    "你好！我叫李华",
    "我想学习 Python 编程",
    "现在几点了？",
    "你还记得我想学什么吗？",
]

print(f"💬 与 {assistant.name} 对话")
print("=" * 50)

for msg in messages:
    print(f"\n👤 用户: {msg}")
    response = assistant.chat(msg, session)
    print(f"🤖 {assistant.name}: {response}")

In [ ]:
# 查看对话历史摘要
print("\n📜 对话历史摘要:")
print("-" * 50)
print(assistant.get_history_summary(session))

---
## 8. 练习

In [ ]:
# 练习 1: 创建一个新的对话会话
# 尝试告诉助手一些关于你的信息，然后测试它是否记得

my_session = "my_session_001"  # 你的会话 ID

# 告诉助手一些信息
print("👤 用户: 我叫小王，我在深圳工作")
response = assistant.chat("我叫小王，我在深圳工作", my_session)
print(f"🤖 小助: {response}")

In [ ]:
# 继续对话，测试记忆
print("\n👤 用户: 我住在哪里？")
response = assistant.chat("我住在哪里？", my_session)
print(f"🤖 小助: {response}")

In [ ]:
# 结合工具使用
print("\n👤 用户: 查一下我所在城市的天气")
response = assistant.chat("查一下我所在城市的天气", my_session)
print(f"🤖 小助: {response}")

In [ ]:
# 练习 2: 自由对话
# 尝试与助手进行更多对话

your_message = "你能帮我做什么？"  # 修改这里

print(f"\n👤 用户: {your_message}")
response = assistant.chat(your_message, my_session)
print(f"🤖 小助: {response}")

---
## 📝 本课小结

| 概念 | 说明 |
|------|------|
| `Checkpointer` | 保存图执行状态的机制 |
| `MemorySaver` | 内存存储，适合开发和测试 |
| `thread_id` | 会话标识，相同 ID 共享对话历史 |
| `config` | 配置对象，传入 thread_id |
| `get_state` | 获取当前会话的完整状态 |

### 关键代码

```python
# 1. 创建 MemorySaver
memory = MemorySaver()

# 2. 编译时传入 checkpointer
agent = create_react_agent(
    model=model,
    tools=tools,
    checkpointer=memory  # 关键！
)

# 3. 调用时传入 thread_id
config = {"configurable": {"thread_id": "user_123"}}
result = agent.invoke({"messages": [...]}, config=config)
```

---

## 🎉 恭喜完成全部课程！

你已经学会了：
1. ✅ 连接阿里千问模型
2. ✅ 使用 Prompt Template 和 Chain
3. ✅ 创建和使用工具
4. ✅ 使用 LangGraph 构建智能体
5. ✅ 为智能体添加记忆功能

### 下一步建议

1. **尝试更多工具**：连接真实的 API（天气、搜索、数据库等）
2. **持久化存储**：使用 `SqliteSaver` 或 `PostgresSaver`
3. **自定义工作流**：构建更复杂的多步骤智能体
4. **部署应用**：将智能体部署为 Web 服务

祝你在 AI 应用开发的道路上越走越远！🚀